<a href="https://colab.research.google.com/github/tasmita0131/Tasmita-Tanjim/blob/main/CE305_HW005_19723_Tasmita_Tanjim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ANSWER OF 1

def store(storage_dict, data):
    # Adds or updates data in the given storage dictionary
    storage_dict.update(data)
    return storage_dict

def imm_load_ac(value):
    # Directly returns the provided value (immediate addressing mode)
    return value

def dir_load_ac(data_storage, address):
    # Retrieves a value from data_storage using the provided address (direct addressing mode)
    return data_storage[address]

def indir_load_ac(data_storage, pointer):
    # Retrieves the address stored at 'pointer' and then fetches the value from that address (indirect addressing mode)
    actual_address = data_storage[pointer]
    return data_storage[actual_address]

def idx_load_ac(data_storage, index, base):
    # Calculates the final address by adding base and index, then fetches the value (indexed addressing mode)
    final_address = base + index
    return data_storage.get(final_address, None)

# Initializing and populating a new memory storage
memory_store = {}
memory_store = store(memory_store, {800: 123})    # memory_store = {800: 123}
memory_store = store(memory_store, {900: 1000})   # memory_store = {800: 123, 900: 1000}
memory_store = store(memory_store, {800: 900})    # memory_store = {800: 900, 900: 1000}
memory_store = store(memory_store, {1500: 700})   # memory_store = {800: 900, 900: 1000, 1500: 700}

# Loading values into accumulator using different addressing modes
accumulator_imm = imm_load_ac(800)                  # accumulator = 800
accumulator_dir = dir_load_ac(memory_store, 800)    # accumulator = 900
accumulator_indir = indir_load_ac(memory_store, 800) # accumulator = 1000
index_value = 700
accumulator_idx = idx_load_ac(memory_store, index_value, 800)  # accumulator = None


# Enhanced print statements for output
print("Accumulator Value (Immediate Addressing):",accumulator_imm)
print("Accumulator Value (Direct Addressing):",accumulator_dir)
print("Accumulator Value (Indirect Addressing):",accumulator_indir)
print(f"Accumulator Value (Indexed Addressing): {'Value not found' if accumulator_idx is None else accumulator_idx}")


Accumulator Value (Immediate Addressing): 800
Accumulator Value (Direct Addressing): 900
Accumulator Value (Indirect Addressing): 1000
Accumulator Value (Indexed Addressing): 700


In [4]:
#ANSWER OF 2

# Function to update the contents of the main memory.
# It takes the existing memory dictionary and a new data dictionary to update the memory.
def update_main_memory(memory, data):
    memory.update(data)  # Merge the new data into the existing memory.
    return memory

# Function to map a block of data from main memory into the cache.
# It updates the cache with the data block corresponding to the given address.
def map_block_to_cache(cache, address, main_memory):
    # Split the address into tag, block, and offset parts.
    tag, block, offset = address[:7], address[7:11], address[11:]
    # Retrieve the corresponding block data from main memory.
    data = main_memory.get(address[:11] + '000', None)

    # If the data exists, update the cache with the tag, data, and set the valid bit to 1.
    if data is not None:
        cache[block] = [tag, data, 1]

    return cache

# Function to determine if an address results in a cache hit or miss.
# It compares the tag of the given address with the tag in the cache.
def check_cache_hit_or_miss(cache, address):
    # Extract the tag, block, and offset from the address.
    tag, block, offset = address[:7], address[7:11], address[11:]
    # Retrieve the cache entry for the block.
    cache_entry = cache.get(block, None)

    # Check if the cache entry is valid and the tags match to determine a hit or miss.
    if cache_entry and cache_entry[0] == tag and cache_entry[2] == 1:
        return "Cache Hit"
    else:
        return "Cache Miss"

# Example usage of the functions to demonstrate cache mapping and hit/miss checking.

# Initialize an empty main memory.
initial_memory = {}
# Update main memory with sample data blocks.
main_memory = update_main_memory(initial_memory, {"00000110101000": [0, 1, 2, 3, 4, 5, 6, 7]})
main_memory = update_main_memory(main_memory, {"00001110101000": [10, 11, 12, 13, 14, 15, 16, 17]})
main_memory = update_main_memory(main_memory, {"00001110111000": [20, 21, 22, 23, 24, 25, 26, 27]})

# Initialize the cache with default values.
cache = {f"{i:04b}": ["0000000", [0, 0, 0, 0, 0, 0, 0, 0], 0] for i in range(16)}

# Define sample addresses to be checked.
address1 = "00000110101010"  # Address 1AA in Hex
address2 = "00001110101010"  # Address 3AA in Hex
address3 = "00001110111111"  # Address 3BF in Hex

# Map data to cache and check the cache hit/miss status for each address.
cache = map_block_to_cache(cache, address2, main_memory)
cache = map_block_to_cache(cache, address1, main_memory)
cache = map_block_to_cache(cache, address3, main_memory)

hit_miss1 = check_cache_hit_or_miss(cache, address1)
hit_miss2 = check_cache_hit_or_miss(cache, address2)
hit_miss3 = check_cache_hit_or_miss(cache, address3)

# Print the cache status for each address.
print("Cache Status for Address 1AA:", hit_miss1)
print("Cache Status for Address 3AA:", hit_miss2)
print("Cache Status for Address 3BF:", hit_miss3)


Cache Status for Address 1AA: Cache Hit
Cache Status for Address 3AA: Cache Miss
Cache Status for Address 3BF: Cache Hit


In [ ]:
#ANSWER OF 3

def store(storage, elm):
    """
    Updates the storage with the provided element(s).
    Args:
    storage: The storage dictionary to be updated.
    elm: The element(s) to be added or updated in the storage.
    Returns:
    Updated storage dictionary.
    """
    storage.update(elm)
    return storage

def fully_assoc_cache_map(cache, mem, address):

    tag, word = address[:11], address[11:]  # Splitting the address into tag and word parts

    # Check for the block in the cache
    for block in cache.values():
        if block[0] == tag and block[2] == 1:  # If tag matches and valid bit is set
            return cache  # Block is already in the cache

    # Find an empty cache line
    for block_label, block_content in cache.items():
        if block_content[2] == 0:  # If valid bit is 0, indicating empty line
            cache[block_label] = [tag, mem[address], 1]  # Set valid bit to 1
            return cache

    # Replace the first block if no empty line is found
    first_block_label = list(cache.keys())[0]
    cache[first_block_label] = [tag, mem[address], 1]

    return cache

# Initialize main memory and store data
init_mem = {}
mem = store(init_mem, {"00000110101000": [0, 1, 2, 3, 4, 5, 6, 7]})
mem = store(mem, {"00001110101000": [10, 11, 12, 13, 14, 15, 16, 17]})
mem = store(mem, {"00011110101000": [20, 21, 22, 23, 24, 25, 26, 27]})
mem = store(mem, {"00111110101000": [30, 31, 32, 33, 34, 35, 36, 37]})
mem = store(mem, {"01111110101000": [40, 41, 42, 43, 44, 45, 46, 47]})

# Initialize cache with empty blocks
cache = {
    "blk0": ["00000000000", [0, 0, 0, 0, 0, 0, 0, 0], 0],
    "blk1": ["00000000000", [0, 0, 0, 0, 0, 0, 0, 0], 0],
    "blk2": ["00000000000", [0, 0, 0, 0, 0, 0, 0, 0], 0],
    "blk3": ["00000000000", [0, 0, 0, 0, 0, 0, 0, 0], 0]
}

# Mapping blocks from memory to cache
addresses = ["00000110101000", "00001110101000", "00011110101000", "00111110101000", "01111110101000"]

#using a for loop
for addr in addresses:
    cache = fully_assoc_cache_map(cache, mem, addr)

# Enhanced print statements for the cache state
print("Current Cache Configuration:")
for block_label, block_content in cache.items():
    print(f"{block_label}: Tag: {block_content[0]}, Data: {block_content[1]}, Valid Bit: {block_content[2]}")


Current Cache Configuration:
blk0: Tag: 01111110101, Data: [40, 41, 42, 43, 44, 45, 46, 47], Valid Bit: 1
blk1: Tag: 00001110101, Data: [10, 11, 12, 13, 14, 15, 16, 17], Valid Bit: 1
blk2: Tag: 00011110101, Data: [20, 21, 22, 23, 24, 25, 26, 27], Valid Bit: 1
blk3: Tag: 00111110101, Data: [30, 31, 32, 33, 34, 35, 36, 37], Valid Bit: 1
